<a href="https://colab.research.google.com/github/Feranie/Hierarchical-Classification-Project/blob/main/hierarchical_global_model_naives_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importação das bibliotecas necessárias
import numpy as np  # Biblioteca para operações numéricas e arrays
import os  # Biblioteca para operações do sistema operacional
from sklearn.datasets import load_iris  # Importa dataset iris do sklearn (não usado no código)
from collections import defaultdict  # Importa defaultdict para criar dicionários com valores padrão

class ArffReader:
    """Classe para ler arquivos no formato ARFF (Attribute-Relation File Format)"""

    def __init__(self):
        """Construtor da classe - inicializa listas vazias para atributos e dados"""
        self.attributes = []  # Lista para armazenar os nomes dos atributos
        self.data = []  # Lista para armazenar os dados/instâncias

    def read_arff(self, input_file):
        """Método para ler um arquivo ARFF e extrair atributos e dados"""
        reading_data = False  # Flag para indicar se estamos lendo a seção de dados
        try:
            # Abre o arquivo em modo leitura com codificação UTF-8
            with open(input_file, 'r', encoding='utf-8') as file:
                # Itera sobre cada linha do arquivo
                for line in file:
                    line = line.strip()  # Remove espaços em branco no início e fim da linha
                    # Pula linhas vazias ou comentários (que começam com %)
                    if not line or line.startswith('%'):
                        continue
                    # Se a linha contém '@attribute', extrai o nome do atributo
                    if '@attribute' in line.lower():
                        attr_name = line.split()[1]  # Pega o segundo elemento (nome do atributo)
                        self.attributes.append(attr_name)  # Adiciona à lista de atributos
                        continue
                    # Se encontrou '@data', marca que começamos a ler os dados
                    if '@data' in line.lower():
                        reading_data = True
                        continue
                    # Se estamos na seção de dados, processa a linha
                    if reading_data:
                        self.data.append(line.split(','))  # Divide a linha por vírgulas e adiciona aos dados
            # Imprime informações sobre o arquivo lido
            print(f"Total d'attributs : {len(self.attributes)}")
            print(f"Total d'instances : {len(self.data)}")
        except Exception as e:
            # Se houver erro, imprime a mensagem e relança a exceção
            print(f"Erreur lors de la lecture du fichier : {str(e)}")
            raise


def build_ancestors(labels):
    """Função para construir um dicionário de ancestrais para cada label na hierarquia"""
    ancestors = defaultdict(list)  # Cria dicionário com lista vazia como valor padrão
    # Para cada par de labels, verifica se um é ancestral do outro
    for label1 in labels:
        for label2 in labels:
            # Se label2 está contido em label1, então label2 é ancestral de label1
            if label2 in label1:
                ancestors[label1].append(label2)  # Adiciona label2 como ancestral de label1
    return dict(ancestors)  # Converte defaultdict para dict normal

class HierarchicalGlobalNaivesBayes:
    """Classe que implementa o algoritmo Naive Bayes Hierárquico Global"""

    def __init__(self, hierarchy, alpha=1):
        """Construtor da classe"""
        # Constrói a hierarquia completa a partir da hierarquia fornecida
        self.labels = self.build_complet_hierarchy(hierarchy)
        self.alpha = alpha  # Parâmetro de suavização de Laplace
        self.descendants = {}  # Dicionário para armazenar descendentes de cada classe
        self.ancestors = {}  # Dicionário para armazenar ancestrais de cada classe


    def build_complet_hierarchy(self, hierarchy):
        """Método para construir a hierarquia completa incluindo todos os níveis"""
        complet_hierarchy = []  # Lista para armazenar a hierarquia completa
        # Para cada classe na hierarquia original
        for classe in hierarchy:
            # Para cada nível da classe (separado por pontos)
            for _ in range(len(classe.split('.'))):
                # Se a classe ainda não está na hierarquia completa, adiciona
                if classe not in complet_hierarchy:
                    complet_hierarchy.append(classe)
                    # Remove o último nível da classe (vai subindo na hierarquia)
                    classe = classe.split('.')[:-1]
                    # Se sobrou apenas um elemento (raiz), para o loop
                    if len(classe) == 1: # Sobrou so o R
                        break
                    # Reconstrói a string da classe
                    classe = '.'.join(classe)
        return complet_hierarchy


    def build_descendants(self, labels):
        """Método para construir um dicionário de descendentes para cada label"""
        descendants = defaultdict(list)  # Cria dicionário com lista vazia como valor padrão
        # Para cada par de labels, verifica relação de descendência
        for label1 in labels:
            for label2 in labels:
                # Se label2 está contido em label1, então label1 é descendente de label2
                if label2 in label1:
                    descendants[label2].append(label1)  # Adiciona label1 como descendente de label2
        return dict(descendants)  # Converte para dict normal

    def fit(self, X, y):
        """Método para treinar o modelo com os dados de treino"""
        n_samples, n_features = X.shape  # Obtém número de amostras e características
        # Constrói dicionários de descendentes e ancestrais
        self.descendants = self.build_descendants(self.labels)
        self.ancestors = build_ancestors(self.labels)

        # Lista com um dicionário para cada feature para armazenar probabilidades
        self.feature_probs = [{} for _ in range(n_features)]
        # Dicionário para armazenar probabilidades a priori de cada classe
        self.prior_prob = {classe: 0 for classe in self.labels}

        # Número de valores únicos por atributo, usado na suavização de Laplace
        self.n_values_per_att = {}
        for feature_idx in range(n_features):
            self.n_values_per_att[feature_idx] = len(np.unique(X[:, feature_idx]))

        # Número de ocorrências de cada classe, usado na suavização de Laplace
        self.n_class_occurances = {}
        classe, counts = np.unique(y, return_counts=True)  # Conta ocorrências de cada classe
        for c, count in zip(classe, counts):
            self.n_class_occurances[c] = count  # Armazena contagem para cada classe

        # Para cada classe nos labels
        for idx, c in enumerate(self.labels):
            # Seleciona as instâncias que pertencem à classe atual
            X_c = X[y == c]
            n_instancias_classe_c = X_c.shape[0]  # Número de instâncias da classe c

            # Para cada feature, calcula as probabilidades condicionais
            for feature_idx in range(n_features):
                # Obtém valores únicos e suas contagens para esta feature
                feature_vals, counts = np.unique(X_c[:, feature_idx], return_counts=True)
                n_valores_unicos_feat = len(np.unique(X_c[:, feature_idx]))

                # Calcula probabilidade P(feature_val|classe) usando suavização de Laplace
                # Armazena em um dicionário pares (valor do atributo, probabilidade)
                feature_prob = {val: (count + self.alpha) / (n_instancias_classe_c + self.alpha * n_valores_unicos_feat) for val, count in zip(feature_vals, counts)}

                # Armazena as probabilidades desta feature para esta classe
                self.feature_probs[feature_idx][c] = feature_prob

            # Soma a contagem da classe atual em todos os seus ancestrais
            for classe in self.ancestors[c]:
                self.prior_prob[classe] += n_instancias_classe_c + self.alpha

        # Normaliza as probabilidades a priori usando a fórmula de Laplace
        for classe in self.prior_prob:
            self.prior_prob[classe] /= (n_samples + (self.alpha * len(self.labels)))

    def calculate_usefullness(self):
        """Método para calcular a utilidade de cada classe baseada no tamanho da árvore"""
        # Encontra o tamanho máximo da árvore (maior número de descendentes)
        max_tree_size = max([len(value) for key, value in self.descendants.items()])
        usefullness = []  # Lista para armazenar utilidades
        # Para cada classe, calcula sua utilidade
        for c in self.labels:
            tree_size_i = len(self.descendants[c])  # Tamanho da árvore para esta classe
            # Fórmula da utilidade: 1 - (log2(tamanho_arvore) / tamanho_max_arvore)
            usefullness_i = 1 - (np.log2(tree_size_i) / max_tree_size)
            usefullness.append(usefullness_i)  # Adiciona à lista
        return usefullness

    def predict(self, X_test, usefullness=False):
        """Método para fazer predições em dados de teste"""
        # Se deve usar fator de utilidade, calcula as utilidades
        if usefullness:
            usefullness_list = self.calculate_usefullness()
        X_test = X_test.astype(np.float64)  # Converte dados de teste para float64
        predictions = []  # Lista para armazenar predições

        # Para cada instância que se deseja predizer
        for x in X_test:
            posteriors = []  # Lista para armazenar probabilidades posteriores
            # Para cada classe possível
            for idx, c in enumerate(self.labels):
                # Calcula log(P(c)) + log(P(x1|c)) + log(P(x2|c)) + ... + log(P(xn|c))
                # Obtém a probabilidade a priori da classe (em log)
                prior = np.log(self.prior_prob[c])

                likelihood = 0  # Inicializa likelihood
                # Para cada feature da instância
                for feature_idx, feature_val in enumerate(x):
                    feature_val = int(feature_val)  # Converte para inteiro
                    # Verifica se este valor da feature existe para esta classe
                    if feature_val in self.feature_probs[feature_idx][c]:
                        # Soma o log da probabilidade P(feature_val|classe)
                        likelihood += np.log(self.feature_probs[feature_idx][c][feature_val])
                    else:
                        # Se não existe, usa suavização de Laplace para estimar
                        nvals = float(self.n_values_per_att[feature_idx])
                        likelihood += np.log(self.alpha / (self.n_class_occurances.get(c, 0) + nvals*self.alpha))

                # Calcula probabilidade posterior: prior + likelihood
                posterior = prior + likelihood
                # Se deve usar utilidade, adiciona o log da utilidade
                if usefullness:
                    posterior += np.log(usefullness_list[idx])
                posteriors.append(posterior)  # Adiciona à lista de posteriores

            # Escolhe a classe com maior probabilidade posterior
            # np.argmax retorna o índice que maximiza o array "posteriors"
            predictions.append(self.labels[np.argmax(posteriors)])

        return np.array(predictions)  # Retorna array numpy com as predições

def metrics_hierarquica(predictions, y_true, labels):
    """Função para calcular métricas hierárquicas (precisão, recall e F1)"""
    ancestors = build_ancestors(labels)  # Constrói dicionário de ancestrais
    numerador = 0  # Numerador para precisão e recall
    denominador_precision = 0  # Denominador para precisão
    denominador_recall = 0  # Denominador para recall

    # Para cada par (predição, valor verdadeiro)
    for classe_predita, classe_verdadeira in zip(predictions, y_true):
        # Obtém ancestrais da classe predita e verdadeira
        ancestrais_classe_predita = ancestors.get(classe_predita, [])
        ancestrais_classe_verdadeira = ancestors.get(classe_verdadeira, [])
        classes_comum = 0  # Conta classes em comum

        # Conta quantas classes ancestrais são comuns
        for c1 in ancestrais_classe_predita:
            for c2 in ancestrais_classe_verdadeira:
                if c1 == c2:
                    classes_comum += 1

        # Acumula valores para cálculo das métricas
        numerador += classes_comum
        denominador_precision += len(ancestrais_classe_predita)
        denominador_recall += len(ancestrais_classe_verdadeira)

    # Calcula precisão hierárquica
    hierarchical_precision = numerador / denominador_precision if denominador_precision != 0 else 0
    # Calcula recall hierárquico
    hierarchical_recall = numerador / denominador_recall if denominador_recall != 0 else 0

    # Calcula F1-score hierárquico
    if hierarchical_precision + hierarchical_recall == 0:
        f_measure = 0
    else:
        f_measure = (2 * hierarchical_precision * hierarchical_recall) / (hierarchical_precision + hierarchical_recall)

    return hierarchical_precision, hierarchical_recall, f_measure


def main(train_file_path, test_file_path, alpha):
    """Função principal que executa todo o pipeline de treinamento e teste"""
    print("📖 Lecture du fichier d'entraînement...")
    # Cria leitor para arquivo de treino
    train_reader = ArffReader()
    train_reader.read_arff(train_file_path)  # Lê arquivo de treino

    # Converte dados de treino para array numpy
    # Remove última coluna (rótulo) e converte para float, substituindo '?' por 0.0
    X_train = np.array([
        list(map(lambda x: float(x) if x and x != '?' else 0.0, row[:-1]))
        for row in train_reader.data
    ])
    # Extrai rótulos (última coluna) como array numpy
    y_train = np.array([row[-1] for row in train_reader.data])

    print("📖 Lecture du fichier de test...")
    # Cria leitor para arquivo de teste
    test_reader = ArffReader()
    test_reader.read_arff(test_file_path)  # Lê arquivo de teste

    # Converte dados de teste para array numpy (mesmo processo que treino)
    X_test = np.array([
        list(map(lambda x: float(x) if x and x != '?' else 0.0, row[:-1]))
        for row in test_reader.data
    ])
    # Extrai rótulos de teste
    y_test = np.array([row[-1] for row in test_reader.data])

    # Imprime informações sobre os dados
    print(f"\n=== Test avec alpha = {alpha} ===")
    print(f"📊 Données d'entraînement: {len(X_train)} instances")
    print(f"📊 Données de test: {len(X_test)} instances")

    # Obtém hierarquia única dos rótulos de treino
    hierarchy = np.unique(y_train)
    print(f"Hiérarchie détectée: {len(hierarchy)} classes")

    # Cria e treina o modelo
    model = HierarchicalGlobalNaivesBayes(hierarchy, alpha=alpha)
    print("Entraînement du modèle...")
    model.fit(X_train, y_train)  # Treina o modelo

    # Faz predições sem fator de utilidade
    print("\n🔍 Résultats sans facteur d'utilité:")
    y_pred = model.predict(X_test, usefullness=False)

    # Calcula métricas hierárquicas
    hierarchical_precision, hierarchical_recall, f1 = metrics_hierarquica(y_pred, y_test, model.labels)

    # Imprime resultados
    #print(f"hierarchical_precision (hp): {hierarchical_precision * 100:.2f}%")
    #print(f"hierarchical_recall (hR): {hierarchical_recall * 100:.2f}%")
    print(f"F1-score hiérarchique (hF): {f1 * 100:.2f}%")


# Bloco principal - executa apenas se o script for executado diretamente
if __name__ == "__main__":
    # Define caminhos dos arquivos de treino e teste
    train_file_path = "/content/GPCR-PfamTRA0.arff"
    test_file_path = "/content/GPCR-PfamTES0.arff"
    # Testa com alpha = 1.0
    for alpha in [1.0,0.01,0.1]:
        main(train_file_path, test_file_path, alpha)

📖 Lecture du fichier d'entraînement...
Erreur lors de la lecture du fichier : [Errno 2] No such file or directory: '/content/GPCR-PfamTRA0.arff'


FileNotFoundError: [Errno 2] No such file or directory: '/content/GPCR-PfamTRA0.arff'